In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
from functools import partial

import torch.nn.functional as F
from torch import nn

import emmental
from emmental import Meta
from emmental.learner import EmmentalLearner
from emmental.model import EmmentalModel
from emmental.scorer import Scorer
from emmental.task import EmmentalTask
from modules.bert_module import BertModule
from parse_CB import get_CB_dataloaders
from task_config import SuperGLUE_LABEL_MAPPING, SuperGLUE_TASK_METRIC_MAPPING
from sklearn.metrics import f1_score

In [3]:
logger = logging.getLogger(__name__)

# Initalize Emmental

In [4]:
emmental.init(
    "logs",
    config={
        "meta_config": {"device": 1, "seed": 111},
        "learner_config": {
            "n_epochs": 20,
            "valid_split": "val",
            "optimizer_config": {"optimizer": "adam", "lr": .00001},
            "min_lr": 0,
            "lr_scheduler_config": {
                "warmup_percentage": 0.1,
                "lr_scheduler": "linear",
            },
        },
        "logging_config": {
            "counter_unit": "epoch",
            "evaluation_freq": 1,
            "checkpointing": None,
        },
    },
)

05/19/2019 22:02:40 - INFO - emmental.meta -   Setting logging directory to: logs/2019_05_19/22_02_40
05/19/2019 22:02:40 - INFO - emmental.meta -   Loading Emmental default config from /dfs/scratch1/senwu/mmtl/emmental/src/emmental/emmental-default-config.yaml.
05/19/2019 22:02:40 - INFO - emmental.meta -   Updating Emmental config from user provided config.


In [5]:
Meta.config

{'meta_config': {'seed': 111, 'verbose': True, 'log_path': None, 'device': 1},
 'model_config': {'model_path': None, 'device': 0},
 'learner_config': {'fp16': False,
  'n_epochs': 20,
  'train_split': 'train',
  'valid_split': 'val',
  'test_split': 'test',
  'ignore_index': -100,
  'optimizer_config': {'optimizer': 'adam',
   'lr': 1e-05,
   'l2': 0.0,
   'grad_clip': 1.0,
   'sgd_config': {'momentum': 0.9},
   'adam_config': {'betas': (0.9, 0.999)}},
  'lr_scheduler_config': {'lr_scheduler': 'linear',
   'warmup_steps': None,
   'warmup_unit': 'batch',
   'warmup_percentage': 0.1,
   'min_lr': 0.0,
   'linear_config': {'min_lr': 0.0},
   'exponential_config': {'gamma': 0.9},
   'plateau_config': {'factor': 0.5, 'patience': 10, 'threshold': 0.0001}},
  'task_scheduler': 'round_robin',
  'global_evaluation_metric_dict': None,
  'min_lr': 0},
 'logging_config': {'counter_unit': 'epoch',
  'evaluation_freq': 1,
  'writer_config': {'writer': 'tensorboard', 'verbose': True},
  'checkpointi

In [6]:
TASK_NAME = "CB"
DATA_DIR = "data"
BERT_MODEL_NAME = "bert-large-cased"
BATCH_SIZE = 4

# Extract train/dev dataset from file

In [7]:
dataloaders = get_CB_dataloaders(
    data_dir=DATA_DIR,
    task_name=TASK_NAME,
    splits=["train", "val"],
    max_sequence_length=256,
    max_data_samples=None,
    tokenizer_name=BERT_MODEL_NAME,
    batch_size=BATCH_SIZE,
)

05/19/2019 22:02:40 - INFO - tokenizer -   Loading Tokenizer bert-large-cased
05/19/2019 22:02:40 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-vocab.txt from cache at /lfs/local/0/senwu/.pytorch_pretrained_bert/cee054f6aafe5e2cf816d2228704e326446785f940f5451a5b26033516a4ac3d.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


data/CB/train.jsonl
{'premise': 'It was a complex language. Not written down but handed down. One might say it was peeled down.', 'hypothesis': 'the language was peeled down', 'label': 'entailment', 'idx': 0}


05/19/2019 22:02:41 - INFO - parse_CB -   Loaded train for CB.
05/19/2019 22:02:41 - INFO - parse_CB -   Loaded val for CB.


data/CB/val.jsonl
{'premise': "Valence the void-brain, Valence the virtuous valet. Why couldn't the figger choose his own portion of titanic anatomy to shaft? Did he think he was helping?", 'hypothesis': 'Valence was helping', 'label': 'contradiction', 'idx': 0}


# Build Emmental task

In [8]:
def ce_loss(task_name, immediate_ouput_dict, Y, active):
    module_name = f"{task_name}_pred_head"
    return F.cross_entropy(
        immediate_ouput_dict[module_name][0][active], (Y.view(-1) - 1)[active]
    )

In [9]:
def output(task_name, immediate_ouput_dict):
    module_name = f"{task_name}_pred_head"
    return F.softmax(immediate_ouput_dict[module_name][0], dim=1)

In [10]:
def macro_f1(golds, probs, preds):
    return {"macro_f1": f1_score(golds, preds, average="macro")}

In [11]:
BERT_OUTPUT_DIM = 768 if "uncased" in BERT_MODEL_NAME else 1024
TASK_CARDINALITY = (
    len(SuperGLUE_LABEL_MAPPING[TASK_NAME].keys())
    if SuperGLUE_LABEL_MAPPING[TASK_NAME] is not None
    else 1
)

emmental_task = EmmentalTask(
    name=TASK_NAME,
    module_pool=nn.ModuleDict(
        {
            "bert_module": BertModule(BERT_MODEL_NAME),
            f"{TASK_NAME}_pred_head": nn.Linear(BERT_OUTPUT_DIM, TASK_CARDINALITY),
        }
    ),
    task_flow=[
        {
            "name": "input",
            "module": "bert_module",
            "inputs": [("_input_", "token_ids"), ("_input_", "token_segments")],
        },
        {
            "name": f"{TASK_NAME}_pred_head",
            "module": f"{TASK_NAME}_pred_head",
            "inputs": [("input", 1)],
        },
    ],
    loss_func=partial(ce_loss, TASK_NAME),
    output_func=partial(output, TASK_NAME),
    scorer=Scorer(
        metrics=SuperGLUE_TASK_METRIC_MAPPING[TASK_NAME],
        customize_metric_funcs={"macro_f1": macro_f1},
    ),
)

05/19/2019 22:02:41 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased.tar.gz from cache at ./cache/7fb0534b83c42daee7d3ddb0ebaa81387925b71665d6ea195c5447f1077454cd.eea60d9ebb03c75bb36302aa9d241d3b7a04bba39c360cf035e8bf8140816233
05/19/2019 22:02:41 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/7fb0534b83c42daee7d3ddb0ebaa81387925b71665d6ea195c5447f1077454cd.eea60d9ebb03c75bb36302aa9d241d3b7a04bba39c360cf035e8bf8140816233 to temp dir /tmp/tmpjpi9j1mq
05/19/2019 22:02:59 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads"

In [12]:
mtl_model = EmmentalModel(name="SuperGLUE_single_task", tasks=[emmental_task])

05/19/2019 22:03:34 - INFO - emmental.model -   Moving model to GPU (cuda:0).
05/19/2019 22:03:46 - INFO - emmental.model -   Created emmental model SuperGLUE_single_task that contains task {'CB'}.
05/19/2019 22:03:46 - INFO - emmental.model -   Moving model to GPU (cuda:0).


In [13]:
emmental_learner = EmmentalLearner()

In [14]:
emmental_learner.learn(mtl_model, dataloaders.values())

05/19/2019 22:03:46 - INFO - emmental.logging.logging_manager -   Evaluating every 1 epoch.
05/19/2019 22:03:46 - INFO - emmental.logging.logging_manager -   No checkpointing.
05/19/2019 22:03:46 - INFO - root -   Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
05/19/2019 22:03:46 - INFO - root -   Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
05/19/2019 22:03:46 - INFO - emmental.learner -   Warmup 126 batchs.
05/19/2019 22:03:46 - INFO - emmental.learner -   Start learning...


/lfs/raiders3/0/senwu/.venv_emmental/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
mtl_model.score(dataloaders["val"])

{'CB/SuperGLUE/val/accuracy': 0.8571428571428571,
 'CB/SuperGLUE/val/macro_f1': 0.8513708513708513}

In [16]:
mtl_model.score(dataloaders["val"])

{'CB/SuperGLUE/val/accuracy': 0.8571428571428571,
 'CB/SuperGLUE/val/macro_f1': 0.8513708513708513}